# Dijkstras Algorithm

- [Problem Setting](#problem-setting)
- [Step-by-Step Explanation](#step-by-step-explanation)
- [Algorithm in Python](#algorithm-in-python)
- [The Algorithm in Action](#the-algorithm-in-action)


Dijkstra’s algorithm is a popular search algorithm used to determine the shortest path between two nodes in a graph.<br>
This algorithm can be applied to any problem that can be represented as a graph, such as but not limited to:
- Friend suggestions on social media
- routing packets over the internet
- finding a way through a maze
etc

The basic runtime of  Dijkstras Algorithm is **O(|V|^2)** (the numer of vertices squared). the worst case we visit every node and in the graph once or twice, and everytime we visit we searc through the queue to find the minimum element.

<hr>

# Problem Setting

The example we’ll use in this notebook for simplicity wull be classic problem of: The shortest path between two cities.<br>
The two cites we will use will be the European cities of Reykjavik and Belgrade.

- Each city is represented as a node.
- Each road is represented as an edge.
- Each road has an associated value. A value could be the distance between cities, a highway toll, or the amount of traffic. Generally, we’ll favor edges with lower values. In our specific case, the associated value is defined by the distance between two cities.

Obviously we cannot access Belgrade from Reykjavik directly, as this would make the problem quite pointless, so there will be several paths that link them:

- Reykjavik –> Oslo –> Berlin –> Belgrade
- Reykjavik –> London –> Berlin –> Rome –> Athens –> Belgrade
- Reykjavik –> London –> Berlin –> Rome –> Athens –> Moscow –> Belgrade

Each of these paths end in Belgrade, but they all have different values. We can use Dijkstra’s algorithm to find the path with the lowest total value.

<hr>

# Step-by-Step Explanation

First, we initialize the algorithm:

- We set Reykjavik as the starting node.
- We set the distances between Reykjavik and all other cities to infinity, except for the distance between Reykjavik and itself, which we set to 0.

After that, we iteratively execute the following steps:

- We choose the node with the smallest value as the “current node” and visit all of its neighboring nodes. As we visit each neighbor, we update their tentative distance from the starting node.
- Once we visit all of the current node’s neighbors and update their distances, we mark the current node as “visited.” Marking a node as “visited” means that we’ve arrived at its final cost.
- We go back to step one. The algorithm loops until it visits all the nodes in the graph. 

In this example, we start by marking Reykjavik as the “current node” since its value is 0. We proceed by visiting Reykjavik’s two neighboring nodes: London and Oslo. At the beginning of the algorithm, their values are set to infinity, but as we visit the nodes, we update the value for London to 4, and Oslo to 5.

We then mark Reykjavik as “visited.” We know that its final cost is zero, and we don’t need to visit it again. We continue with the next node with the lowest value, which is London.

We visit all of London’s neighboring nodes which we haven’t marked as “visited.” London’s neighbors are Reykjavik and Berlin, but we ignore Reykjavik because we’ve already visited it. Instead, we update Berlin’s value by adding the value of the edge connecting London and Berlin (3) to the value of London (4), which gives us a value of 7.


We mark London as visited and choose the next node: Oslo. We visit Oslo’s neighbors and update their values. It turns out that we can better reach Berlin through Oslo (with a value of 6) than through London, so we update its value accordingly. We also update the current value of Moscow from infinity to 8.


We mark Oslo as “visited” and update its final value to 5. Between Berlin and Moscow, we choose Berlin as the next node because its value (6) is lower than Moscow’s (8). We proceed as before: We visit Rome and Belgrade and update their tentative values, before marking Berlin as “visited” and moving on to the next city.


Note that we’ve already found a path from Reykjavik to Belgrade with a value of 15! But is it the best one?


Ultimately, it’s not. We’ll skip the rest of the steps, but you get the drill. The best path turns out to be Reykjavik –> Oslo –> Berlin –> Rome –> Athens –> Belgrade, with a value of 11.

<hr>

# Algorithm in Python

First, we’ll create the Graph `class`. We’ll implement the graph as a Python `dictionary`. The dictionary’s keys will correspond to the cities and its values will correspond to dictionaries that record the distances to other cities in the graph. 

In [1]:
import sys
 
class Graph(object):
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        """
        This method makes sure that the graph is symmetrical. In other words,
        if there's a path from node A to B with a value V, there needs to be a
        path from node B to node A with a value V.
        """
        graph = {}
        for node in nodes:
            graph[node] = {}

        graph.update(init_graph)

        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value
        
        return graph
    
    def get_nodes(self):
        "Returns the nodes of the graph."
        return self.nodes

    def get_outgoing_edges(self, node):
        "Returns the neighbors of a node."
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections

    def value(self, node1, node2):
        "Returns the value of an edge between two nodes."
        return self.graph[node1][node2]

Now we will implement the Dijkstra algorithm. We’ll start by defining the function.

We can start by defining the function `def dijkstra_algorithm(graph, start_node)`<br>
This function takes two arguments: `graph` and `start_node`. graph is an instance of the Graph class that we created in the previous step, whereas start_node is the node from which we’ll start the calculations. We’ll call the `get_nodes()` method to initialize the list of unvisited nodes:<br> `unvisited_nodes = list(graph.get_nodes())`

Next, we’ll create two dicts, `shortest_path` and `previous_nodes`:

- `shortest_path` will store the best-known cost of visiting each city in the graph starting from the start_node. In the beginning, the cost starts at infinity, but we’ll update the values as we move along the graph.
- `previous_nodes` will store the trajectory of the current best known path for each node. For example, if we know the best way to Berlin to be via Oslo, `previous_nodes["Berlin"]` will return “Oslo”, and `previous_nodes["Oslo"]` will return “Reykjavik.” We’ll use this dictionary to backtrace the shortest path.


Now we can start the algorithm. Remember that Dijkstra’s algorithm executes until it visits all the nodes in a graph, so we’ll represent this as a condition for exiting the while-loop.

`while unvisited_nodes:`

Now, the algorithm can start visiting the nodes. The code block below first instructs the algorithm to find the node with the lowest value. 

Once that’s done, the algorithm visits all node’s neighbors that are still unvisited. If the new path to the neighbor is better than the current best path, the algorithm makes adjustments in the shortest_path and previous_nodes dictionaries.

After visiting all of its neighbors, we can mark the current node as “visited”:

`unvisited_nodes.remove(current_min_node)`

At last, we can return the two dictionaries:    

`return previous_nodes, shortest_path`

In [2]:
def dijkstra_algorithm(graph, start_node):
    
    # Initializr the list of unvisited nodes
    unvisited_nodes = list(graph.get_nodes())
    
    """
    Create two dicts:
    
    1- shortest_path store the best-known cost of visiting each city in the graph 
    starting from the start_node. In the beginning the cost starts at infinity,
    and values are updated later
    
    2- previous_node store the trajectory of the current best known path for each
    node. 
    """
    shortest_path = {}
    previous_nodes = {}
    # We'll use max_value to initialize the "infinity" value of the unvisited nodes
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0
    shortest_path[start_node] = 0

    # Dijkstra executes until it visits all the nodes in a graph, so we use while-loop
    while unvisited_nodes:
        
        # this block instructs the algorithm to find the node with the lowest value
        current_min_node = None
        for node in unvisited_nodes: #iterate over the nodes
            # if the new path to the neighbor is better than current best, 
            # the algorithm makes the adjustments
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
        
        # this code block retrieves the current node's neighbors and updates the 
        # distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # we also update teh best path to the current node
                previous_nodes[neighbor] = current_min_node
        
        unvisited_nodes.remove(current_min_node)
        
    # return the two dictionaries
    return previous_nodes, shortest_path

Lastly, we need to create a function that prints out the results. This function will take the two dictionaries returned by the dijskstra_algorithm function, as well as the names of the beginning and target nodes. It’ll use the two dictionaries to find the best path and calculate the path’s score.

In [3]:
def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    # Add the start node manually
    path.append(start_node)

    print("We found the following best path with a value of {}.".format(shortest_path[target_node]))
    print(" -> ".join(reversed(path)))

# The Algorithm in Action

Now, let’s see the algorithm in action. We’ll manually initialize the nodes and their edges.

In [4]:
nodes = ["Reykjavik", "Oslo", "Moscow", "London", "Rome", "Berlin", "Belgrade", "Athens"]

init_graph = {}
for node in nodes:
    init_graph[node] = {}

init_graph["Reykjavik"]["Oslo"] = 5
init_graph["Reykjavik"]["London"] = 4
init_graph["Oslo"]["Berlin"] = 1
init_graph["Oslo"]["Moscow"] = 3
init_graph["Moscow"]["Belgrade"] = 5
init_graph["Moscow"]["Athens"] = 4
init_graph["Athens"]["Belgrade"] = 1
init_graph["Rome"]["Berlin"] = 2
init_graph["Rome"]["Athens"] = 2

We’ll use these values to create an object of the Graph class.

In [5]:
graph = Graph(nodes, init_graph)

With our graph fully constructed, we can pass it to the `dijkstra_algorithm()` function.

In [6]:
previous_nodes, shortest_path = dijkstra_algorithm(graph=graph, start_node="Reykjavik")

And now let’s print out the results:

In [7]:
	
print_result(previous_nodes, shortest_path, start_node="Reykjavik", target_node="Belgrade")

We found the following best path with a value of 11.
Reykjavik -> Oslo -> Berlin -> Rome -> Athens -> Belgrade
